In [ ]:
# ============================================
# Konversi Koordinat Geografis (Lat-Lon) ke UTM
# GUI + Peta Cartopy - Versi Stabil
# ============================================

# Import library utama
import numpy as np, tkinter as tk
from tkinter import Toplevel, Label, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import cartopy.crs as ccrs, cartopy.feature as cfeature

# Konstanta Ellipsoid WGS84 untuk sistem UTM
A, F = 6378137.0, 1/298.257223563
K0, E0 = 0.9996, 500000
N0_S, N0_N = 10000000, 0  # false northing Sel/Utara

# Menentukan nomor zona UTM dari longitude (°)
def zona_utm(lon_deg):
    return int((lon_deg + 180)//6) + 1

# Menentukan hemisfer (Utara/Selatan)
def hemi(lat):
    return "N" if lat >= 0 else "S"

# Validasi batas koordinat UTM
def valid(lat, lon):
    return -80 <= lat <= 84 and -180 <= lon <= 180

# Fungsi utama konversi Lat-Lon → UTM
def ke_utm(lat_deg, lon_deg):

    lat = np.radians(lat_deg)
    lon = np.radians(lon_deg)

    e2 = 2*F - F**2                   # eksentrisitas kuadrat
    Z = zona_utm(lon_deg)             # zona UTM
    lon0 = np.radians((Z-1)*6 - 180 + 3)

    n = A / np.sqrt(1 - e2*np.sin(lat)**2)
    t = np.tan(lat)**2
    c = e2*np.cos(lat)**2/(1-e2)
    A1 = np.cos(lat)*(lon - lon0)     # beda longitude relatif meridian zona

    # Easting
    E = E0 + K0*n*(A1 + (1 - t + c)*A1**3/6)

    # Northing M (meridian arc)
    M = A*((1 - e2/4 - 3*e2**2/64 - 5*e2**3/256)*lat
        - (3*e2/8 + 3*e2**2/32 + 45*e2**3/1024)*np.sin(2*lat)
        + (15*e2**2/256 + 45*e2**3/1024)*np.sin(4*lat)
        - (35*e2**3/3072)*np.sin(6*lat))

    # Northing final (UTM)
    N0 = N0_N if lat_deg >= 0 else N0_S
    N = N0 + K0*(M + n*np.tan(lat)*(A1**2/2 + (5 - t + 9*c)*A1**4/24))

    return E, N, Z, hemi(lat_deg)

# Jalankan konversi dan tampilkan hasil
def hitung():
    try:
        lat, lon = float(e_lat.get()), float(e_lon.get())
        if not valid(lat, lon):
            return messagebox.showerror("Error","Koordinat di luar batas sistem UTM.")

        E, N, Z, H = ke_utm(lat, lon)
        tampil(E, N, Z, H, lat, lon)

    except ValueError:
        messagebox.showerror("Error", "Input harus berupa angka.")

# Tampilkan output + peta cartopy
def tampil(E, N, Z, H, lat, lon):

    win = Toplevel(root)
    Label(win,text=f"Zona UTM: {Z}{H}\nEasting  : {E:.3f}\nNorthing : {N:.3f}",
          font=("Arial", 12)).pack(pady=5)

    fig = plt.Figure(figsize=(6,4))
    ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

    ax.stock_img()
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.gridlines(draw_labels=True, linewidth=0.4)

    ax.plot(lon, lat, "ro", markersize=8, transform=ccrs.PlateCarree())
    ax.set_title("Lokasi Titik pada Peta Dunia (UTM)", fontsize=11)

    canvas = FigureCanvasTkAgg(fig, master=win)
    canvas.get_tk_widget().pack()
    canvas.draw()

# GUI utama
root = tk.Tk()
root.title("Konversi Koordinat ke UTM")

tk.Label(root,text="Latitude (°)").pack()
e_lat = tk.Entry(root); e_lat.pack()

tk.Label(root,text="Longitude (°)").pack()
e_lon = tk.Entry(root); e_lon.pack()

tk.Button(root,text="Konversi UTM",command=hitung).pack(pady=5)

root.mainloop()
